In [27]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import Preprocessing as prep
import Visualization as vis


import warnings
warnings.filterwarnings("ignore")               # 경고 메시지 무시
# warnings.filterwarnings(action='default')     # 경고 메시지 활성화

plt.rcParams['axes.unicode_minus'] = False      # Glyph 8722 오류 해결을 위한 설정
plt.rcParams["figure.figsize"] = (12, 9)        # 그림 크기 설정
plt.rc('font', family="Malgun Gothic")       # 한글 폰트 설정
sns.set_theme(palette="Pastel1")                                 # Seaborn의 테마 자동 설정

train = pd.read_csv("../자료/LG_공정시스템_데이콘/train.csv", engine="python", encoding="utf-8")
test = pd.read_csv("../자료/LG_공정시스템_데이콘/test.csv", engine="python", encoding="utf-8")
submission = pd.read_csv("../자료/LG_공정시스템_데이콘/sample_submission.csv", engine="python", encoding="utf-8")
train

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,A_31,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,A_31,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,TRAIN_593,1,0.526546,2022-09-08 14:30,T100306,T_31,2.0,95.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594,TRAIN_594,0,0.524022,2022-09-08 22:38,T050304,A_31,NaN,NaN,NaN,NaN,...,49.47,53.07,50.89,55.10,66.49,1.0,NaN,NaN,NaN,NaN
595,TRAIN_595,0,0.521289,2022-09-08 22:47,T050304,A_31,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
596,TRAIN_596,1,0.531375,2022-09-08 14:38,T100304,O_31,40.0,94.0,0.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Columns: 2881 entries, PRODUCT_ID to X_2875
dtypes: float64(2876), int64(1), object(4)
memory usage: 13.1+ MB


In [12]:
train.isnull().sum()

PRODUCT_ID      0
Y_Class         0
Y_Quality       0
TIMESTAMP       0
LINE            0
             ... 
X_2871        499
X_2872        598
X_2873        598
X_2874        598
X_2875        598
Length: 2881, dtype: int64

In [46]:
pd.DataFrame(train.isnull().sum() == True)

,0
PRODUCT_ID,False
Y_Class,False
Y_Quality,False
TIMESTAMP,False
LINE,False
...,...
X_2871,False
X_2872,False
X_2873,False
X_2874,False


In [139]:
df_video = pd.read_csv("../자료/chim_video.csv", encoding='utf-8', engine='python')
df_comment = pd.read_csv("../자료/chim_comments.csv", encoding='utf-8', engine='python')

df_comment.drop(df_comment.loc[df_comment.like_count == '2022-07-12T13:57:57Z'].index, axis=0, inplace=True)
df_comment.drop(df_comment.loc[df_comment.like_count == '2021-05-12T01:19:02Z'].index, axis=0, inplace=True)
df_comment.drop(df_comment.loc[df_comment.like_count == '2021-03-04T14:16:28Z'].index, axis=0, inplace=True)
df_comment.drop(df_comment.loc[df_comment.like_count == '2021-02-02T13:43:33Z'].index, axis=0, inplace=True)
df_comment.drop(df_comment.loc[df_comment.like_count == '2020-11-17T10:18:40Z'].index, axis=0, inplace=True)
df_comment.drop(df_comment.loc[df_comment.like_count == '2020-08-29T10:37:41Z'].index, axis=0, inplace=True)
df_comment.drop(df_comment.loc[df_comment.like_count == '2020-05-21T10:15:01Z'].index, axis=0, inplace=True)
df_comment.drop(df_comment.loc[df_comment.like_count == '2018-12-23T23:21:56Z'].index, axis=0, inplace=True)
df_comment.dropna(axis=0, inplace=True)
df_comment.like_count = df_comment.like_count.astype(np.int64)
df_comment

,video_id,comment_id,comment,like_count,created_at,updated_at
0,7QnVBNbTVeM,Ugzmf2KCLNfGMvjOAIx4AaABAg,침소리와. 궤소리의 만남,0,2023-02-27T06:21:37Z,2023-02-27T06:21:37Z
1,7QnVBNbTVeM,UgwcByZZhnHLQtFb_od4AaABAg,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 이날따라 침방장 왜케 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,0,2023-02-27T06:19:49Z,2023-02-27T06:19:49Z
2,7QnVBNbTVeM,UgzK43w3es4_sUES3-p4AaABAg,"오늘 영상을 요약하자면 ""안드로이드는 전기양의 꿈을 꾸는가?""입니다 아아 필립K딕 ...",0,2023-02-27T06:01:43Z,2023-02-27T06:01:43Z
3,7QnVBNbTVeM,UgwTAaUHneZhtpK_LVZ4AaABAg,궤도님 기계가 분명합니다,0,2023-02-27T05:55:00Z,2023-02-27T05:55:00Z
4,7QnVBNbTVeM,Ugxs2qGT1LRd1z0gb9Z4AaABAg,월드컵 특강 너무 좋은데?,0,2023-02-27T05:53:29Z,2023-02-27T05:53:29Z
...,...,...,...,...,...,...
346161,SRS4AeJzhoc,UghVMiKNyVyI6HgCoAEC,아재 게임피지컬 ㅁㅊㄷ ㅁㅊ 답답,0,2016-05-31T10:57:36Z,2016-05-31T10:57:36Z
346162,SRS4AeJzhoc,Ugg8ahjVX6z-13gCoAEC,ㅋㅋㅋㅋㅋㅋ 초반부 설명하는거 웰케 웃겨요,0,2016-05-31T10:39:53Z,2016-05-31T10:39:53Z
346163,SRS4AeJzhoc,UgjYCSAFu6wUQ3gCoAEC,침하,0,2016-05-31T10:39:45Z,2016-05-31T10:39:45Z
346164,SRS4AeJzhoc,UggPw6PSvwpGGngCoAEC,캬 갓겜을 이제서야,0,2016-05-31T10:36:26Z,2016-05-31T10:36:26Z


In [140]:
df_comment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 346133 entries, 0 to 346165
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   video_id    346133 non-null  object
 1   comment_id  346133 non-null  object
 2   comment     346133 non-null  object
 3   like_count  346133 non-null  int64 
 4   created_at  346133 non-null  object
 5   updated_at  346133 non-null  object
dtypes: int64(1), object(5)
memory usage: 18.5+ MB


In [143]:
df_comment.isnull().sum()

video_id      0
comment_id    0
comment       0
like_count    0
created_at    0
updated_at    0
dtype: int64

In [144]:
df_comment.head(2)

,video_id,comment_id,comment,like_count,created_at,updated_at
0,7QnVBNbTVeM,Ugzmf2KCLNfGMvjOAIx4AaABAg,침소리와. 궤소리의 만남,0,2023-02-27T06:21:37Z,2023-02-27T06:21:37Z
1,7QnVBNbTVeM,UgwcByZZhnHLQtFb_od4AaABAg,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 이날따라 침방장 왜케 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,0,2023-02-27T06:19:49Z,2023-02-27T06:19:49Z


In [145]:
df_comment.to_csv("chim_comment.csv", index=False)

In [153]:
df = pd.read_csv("../자료/cctv_df.csv", encoding='utf-8', engine="python")
df.head()

,Unnamed: 0.1,Unnamed: 0,address,state,city,lat,long,install_cnt,date
0,0,0,강원도 고성군 간성읍 신안리 254-18,강원도,고성군,38.379088,128.470971,NaN,2020-07-01
1,1,1,강원도 고성군 간성읍 신안리 254-18,강원도,고성군,38.379088,128.470971,NaN,2020-07-01
2,2,2,강원도 고성군 운봉리,강원도,고성군,38.287864,128.519409,NaN,2020-07-01
3,3,3,강원도 고성군 간성읍 간성로 70,강원도,고성군,38.379515,128.467187,NaN,2020-07-01
4,4,4,강원도 고성군 간성읍 간성로 70,강원도,고성군,38.379515,128.467187,NaN,2020-07-01


In [161]:
new_list = df.state.unique()

new_list[:30]

array(['강원도', '춘천시', '영월군', '단양군', '정선군', '조양동', '교동', '교동로', '금호동',
       '노학동', '대포동', '도문동', '동명동', '선사로1길', '설악동', '양양군', '영랑동', '장사동',
       '조양동/소평로', '조양동/온정로', '조양동/청대로', '중앙동', '청대로232번길', '청학동', '청학로2길',
       '청호동', '중앙로', '양구읍', '동면', '방산면'], dtype=object)